In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import joblib

## Tratamento dos Dados Entrada:

In [2]:
df_entrada = pd.read_csv('dados_entrada.csv', delimiter=',')
df_entrada['DateTime'] = pd.to_datetime(df_entrada['data_abreviada'], format='%Y-%m-%d')
df_entrada = df_entrada.rename(columns={'Disponibilidade': 'disponibilidade_entrada'})
df_entrada.drop(columns=['data_abreviada'], inplace=True)

## Tratamento dos Dados Transporte

In [3]:
df_transporte = pd.read_csv('dados_transporte.csv', delimiter=',')
df_transporte['DateTime'] = pd.to_datetime(df_transporte['data_abreviada'], format='%Y-%m-%d')
df_transporte = df_transporte.rename(columns={'Disponibilidade': 'disponibilidade_transporte'})
df_transporte.drop(columns=['data_abreviada'], inplace=True)

## Tratamento dos Dados Inspeção

In [4]:
df_inspecao = pd.read_csv('dados_inspecao.csv', delimiter=',')
df_inspecao['DateTime'] = pd.to_datetime(df_inspecao['data_abreviada'], format='%Y-%m-%d')
df_inspecao = df_inspecao.rename(columns={'Disponibilidade': 'disponibilidade_inspecao'})
df_inspecao.drop(columns=['data_abreviada'], inplace=True)

## Tratamento dos Dados Limpeza

In [5]:
df_limpeza = pd.read_csv('dados_limpeza.csv', delimiter=',')
df_limpeza['DateTime'] = pd.to_datetime(df_limpeza['data_abreviada'], format='%Y-%m-%d')
df_limpeza = df_limpeza.rename(columns={'Disponibilidade': 'disponibilidade_limpeza'})
df_limpeza.drop(columns=['data_abreviada'], inplace=True)

## Tratamento dos Dados Saída

In [6]:
df_saida = pd.read_csv('dados_saida.csv', delimiter=',')
df_saida['DateTime'] = pd.to_datetime(df_saida['data_abreviada'], format='%Y-%m-%d')
df_saida = df_saida.rename(columns={'Disponibilidade': 'disponibilidade_saida'})
df_saida.drop(columns=['data_abreviada'], inplace=True)

## União dos dados

In [7]:
df_taxa_de_saida = pd.read_csv('dados_taxa_de_saida.csv', delimiter=',')
df_takt_time = pd.read_csv('dados_takt_time.csv', delimiter=',')

In [8]:
df_taxa_de_saida['DateTime'] = pd.to_datetime(df_taxa_de_saida['data_abreviada'], format='%Y-%m-%d')
df_takt_time['DateTime'] = pd.to_datetime(df_taxa_de_saida['data_abreviada'], format='%Y-%m-%d')
df_taxa_de_saida.drop(columns=['data_abreviada'], inplace=True)
df_takt_time.drop(columns=['data_abreviada'], inplace=True)

In [49]:
# df_geral = pd.concat([df_entrada, df_transporte, df_inspecao, df_limpeza, df_saida, df_taxa_de_saida, df_takt_time ], axis=1)
# df_geral = df_geral.reset_index(drop=True)
# df_geral.drop_duplicates(subset='DateTime', keep='first', inplace=True)


In [9]:
df_geral = pd.merge(df_entrada, df_transporte, on='DateTime', how='outer')
df_geral = pd.merge(df_geral, df_inspecao, on='DateTime', how='outer')
df_geral = pd.merge(df_geral, df_limpeza, on='DateTime', how='outer')
df_geral = pd.merge(df_geral, df_saida, on='DateTime', how='outer')
df_geral = pd.merge(df_geral, df_taxa_de_saida, on='DateTime', how='outer')
df_geral = pd.merge(df_geral, df_takt_time, on='DateTime', how='outer')

In [11]:
df_geral['DateTime'] = pd.to_datetime(df_geral['DateTime'])
df_geral['Dia'] = df_geral['DateTime'].dt.day
df_geral['DiaDaSemana'] = df_geral['DateTime'].dt.dayofweek
df_geral['Mes'] = df_geral['DateTime'].dt.month
df_geral['Ano'] = df_geral['DateTime'].dt.year

In [12]:
df_geral.columns

Index(['disponibilidade_entrada', 'DateTime', 'disponibilidade_transporte',
       'disponibilidade_inspecao', 'disponibilidade_limpeza',
       'disponibilidade_saida', 'taxa_de_saida(und/h)', 'takt_time(s/und)',
       'Dia', 'DiaDaSemana', 'Mes', 'Ano'],
      dtype='object')

In [13]:
df_geral

,disponibilidade_entrada,DateTime,disponibilidade_transporte,disponibilidade_inspecao,disponibilidade_limpeza,disponibilidade_saida,taxa_de_saida(und/h),takt_time(s/und),Dia,DiaDaSemana,Mes,Ano
0,0.997916,2022-01-17,0.994998,0.996248,1.000000,0.996665,13.556485,203.305085,17,0,1,2022
1,0.986105,2022-01-18,0.989810,0.993979,0.998610,0.990736,13.228346,202.406250,18,1,1,2022
2,0.995368,2022-01-19,0.999537,1.000000,0.994905,1.000000,13.351878,197.469512,19,2,1,2022
3,0.989810,2022-01-20,0.972209,1.000000,1.000000,1.000000,13.339509,182.966102,20,3,1,2022
4,0.993979,2022-01-21,0.993516,1.000000,0.985178,0.996758,13.228346,188.284884,21,4,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
710,0.995368,2024-04-24,1.000000,0.997684,0.999537,0.998147,13.339509,199.907407,24,2,4,2024
711,0.993979,2024-04-25,1.000000,1.000000,0.999074,1.000000,13.339509,166.932990,25,3,4,2024
712,1.000000,2024-04-26,0.994442,1.000000,0.998610,1.000000,13.228346,190.500000,26,4,4,2024
713,0.000000,2024-04-27,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27,5,4,2024


In [18]:
df_geral.to_csv('df_geral.csv', index=False)

## Criação Regressão Linear

In [14]:
weekday_dummies = pd.get_dummies(df_geral['DiaDaSemana'], prefix='DiaDaSemana')
month_dummies = pd.get_dummies(df_geral['Mes'], prefix='Mes')
df_geral = pd.concat([df_geral, weekday_dummies, month_dummies], axis=1)

In [15]:
# Supondo que 'df' contém as características temporais tratadas e 'Value' é a saída desejada
X = df_geral[['Dia', 'DiaDaSemana_0','DiaDaSemana_1','DiaDaSemana_2','DiaDaSemana_3', 'DiaDaSemana_4', 'DiaDaSemana_5',\
          'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6', 'Mes_7', 'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12']]
y = df_geral['takt_time(s/und)']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
modGerenciadorTakt = LinearRegression()
modGerenciadorTakt.fit(X_train, y_train)
# Salvar o modelo treinado em um arquivo
joblib.dump(modGerenciadorTakt, 'modGerenciadorTakt.pkl')

['modGerenciadorTakt.pkl']

In [19]:
y_pred = modGerenciadorTakt.predict(X_test)
rmse_takt = np.sqrt(mean_squared_error(y_test, y_pred))
print("A raiz quadrada da média do quadrado dos erros do modelo de Regressão Linear é:", rmse_takt)

A raiz quadrada da média do quadrado dos erros do modelo de Regressão Linear é: 11.993234930508564


In [88]:
# Supondo que 'df' contém as características temporais tratadas e 'Value' é a saída desejada
X2 = df_geral[['Dia', 'DiaDaSemana_0','DiaDaSemana_1','DiaDaSemana_2','DiaDaSemana_3', 'DiaDaSemana_4', 'DiaDaSemana_5',\
          'Mes_1', 'Mes_2', 'Mes_3', 'Mes_4', 'Mes_5', 'Mes_6', 'Mes_7', 'Mes_8', 'Mes_9', 'Mes_10', 'Mes_11', 'Mes_12', 'Ano',\
          'disponibilidade_entrada', 'disponibilidade_transporte', 'disponibilidade_inspecao', 'disponibilidade_limpeza', 'disponibilidade_saida']]
y2 = df_geral['taxa_de_saida(und/h)']

# Dividir os dados em conjuntos de treinamento e teste
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [89]:
modGerenciadorTaxaSaida = LinearRegression()
modGerenciadorTaxaSaida.fit(X_train2, y_train2)
# Salvar o modelo treinado em um arquivo
joblib.dump(modGerenciadorTaxaSaida, 'modGerenciadorTaxaSaida.pkl')

['modGerenciadorTaxaSaida.pkl']

In [90]:
y_pred2 = modGerenciadorTaxaSaida.predict(X_test2)
rmse_tsaida = np.sqrt(mean_squared_error(y_test2, y_pred2))
print("A raiz quadrada da média do quadrado dos erros do modelo de Regressão Linear é:", rmse_tsaida)

A raiz quadrada da média do quadrado dos erros do modelo de Regressão Linear é: 0.12502189208244716
